## Configuration

Paste the correct values in the cell below. The **url** should be the URL for the cluster. Do **NOT** include a trailing "/" character, or the "zen" portion of the URL from the Cloud Pak for Data dashboard. For the user credentials, the user should have admin authority for the OpenScale service, as the notebook will be adding machine learning providers to OpenScale.

In [1]:
WOS_CREDENTIALS = {
    "url": "https://zen-cpd-zen.cpd-outcomes-00-57d48117af6edd64ea27fa1142b91b0b-0000.us-south.containers.appdomain.cloud",
    "username": "admin",
    "password": "xxxxxxxxx",
    "version": "3.5"
}

This notebook will deploy one model to a production workspace, and a second model to a pre-production workspace. As the notebook will try and remove each from OpenScale's list of machine learning providers, it is highly recommended that you create a new deployment space for each, as any other models deployed to the same space will be removed from OpenScale monitoring. Deployment spaces can be managed from the [Deployments screen](../../../../../ml-runtime/spaces?context=icp4data).

In [2]:
WML_PRODUCTION_SPACE = '21844281-0655-4de1-af12-700ed36a5d1e'
WML_PREPROD_SPACE = 'c21aa2de-2354-41e4-9e62-1b0d2ee56d85'

Paste the OpenScale datamart ID into the cell below. It can be found by navigating to the [OpenScale dashboard](../../../../../aiopenscale/insights?serviceInstanceNamespace=zen&serviceInstanceDisplayName=OpenScale), navigating to the **Model information** view of any model, and scrolling to the **Integration details** section.

In [3]:
DATAMART_ID = '00000000-0000-0000-0000-1616425950709603'

Paste the names of the deployed models below. They are case-sensitive.

In [4]:
PRODUCTION_DEPLOYMENT = 'predictive attrition - 2018'
PREPROD_DEPLOYMENT = 'predictive attrition - challenger'

## Run the notebook

At this point, you may run all remaining cells of the notebook. When it completes, you can return to the OpenScale dashboard to see the results.

## Authenticate clients

In [5]:
!pip install ibm-cloud-sdk-core --no-cache | tail -n 1
!pip install --upgrade ibm-watson-machine-learning --user | tail -n 1
!pip install --upgrade ibm-watson-openscale --no-cache | tail -n 1

In [6]:
import os
token = os.environ['USER_ACCESS_TOKEN']

WML_CREDENTIALS = {
   "token" : token,
   "instance_id" : "wml_local",
   "url": WOS_CREDENTIALS["url"],
   "version": "3.5"
}

In [7]:
from ibm_watson_openscale import APIClient
from ibm_watson_openscale.utils import *
from ibm_watson_openscale.supporting_classes import *
from ibm_watson_openscale.supporting_classes.enums import *

import json
import requests
import base64
from requests.auth import HTTPBasicAuth
import time

Errors in this next cell are typically caused by using the incorrect URL in the first configuration cell. Ensure that you did not end the URL with a trailing "/" character or include the "zen" portion of the Cloud Pak for Data dashboard URL.

In [8]:
authenticator = CloudPakForDataAuthenticator(
        url=WOS_CREDENTIALS['url'],
        username=WOS_CREDENTIALS['username'],
        password=WOS_CREDENTIALS['password'],
        disable_ssl_verification=True
    )

wos_client = APIClient(service_url=WOS_CREDENTIALS['url'], service_instance_id=DATAMART_ID, authenticator=authenticator)
wos_client.version

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'zen-cpd-zen.cpd-outcomes-00-57d48117af6edd64ea27fa1142b91b0b-0000.us-south.containers.appdomain.cloud'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'zen-cpd-zen.cpd-outcomes-00-57d48117af6edd64ea27fa1142b91b0b-0000.us-south.containers.appdomain.cloud'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


'3.0.6'

In [9]:
from ibm_watson_machine_learning import APIClient

wml_client = APIClient(WML_CREDENTIALS)
wml_client.version

'1.0.66'

## Remove existing spaces as OpenScale machine learning providers

In [10]:
service_providers = wos_client.service_providers.list().result.service_providers
for service_provider in service_providers:
    service_instance_id = service_provider.entity.deployment_space_id
    if service_instance_id == WML_PRODUCTION_SPACE or service_instance_id == WML_PREPROD_SPACE:
        wos_client.service_providers.delete(service_provider.metadata.id)
        print("Deleted existing service_provider for WML instance: {}".format(service_instance_id))

Deleted existing service_provider for WML instance: 21844281-0655-4de1-af12-700ed36a5d1e


## Promote 2018 to deployment space and deploy

## Add production deployment space as OpenScale service provider

In [11]:
SERVICE_PROVIDER_NAME = "Attrition Production Space"
SERVICE_PROVIDER_DESCRIPTION = "Production space for the HR attrition model"

In [12]:
added_service_provider_result = wos_client.service_providers.add(
        name=SERVICE_PROVIDER_NAME,
        description=SERVICE_PROVIDER_DESCRIPTION,
        service_type=ServiceTypes.WATSON_MACHINE_LEARNING,
        deployment_space_id = WML_PRODUCTION_SPACE,
        operational_space_id = "production",
        credentials=WML_CREDENTIALS,
        background_mode=False
    ).result
service_provider_id = added_service_provider_result.metadata.id




 Waiting for end of adding service provider dc12f972-a7d6-4a3e-83aa-994bed18b7e9 




active

-----------------------------------------------
 Successfully finished adding service provider 
-----------------------------------------------




In [13]:
wml_client.set.default_space(WML_PRODUCTION_SPACE)

'SUCCESS'

In [14]:
DEPLOYMENT_UID = ''
deployments_list = wml_client.deployments.get_details()
for deployment in deployments_list["resources"]:
    if deployment['metadata']['name'] == PRODUCTION_DEPLOYMENT:
        DEPLOYMENT_UID = deployment['metadata']['id']
print(DEPLOYMENT_UID)

6b8d6961-16d2-4c1d-9389-be2c300e8bb7


In [15]:
asset_deployment_details = wos_client.service_providers.list_assets(data_mart_id=DATAMART_ID, service_provider_id=service_provider_id, deployment_id=DEPLOYMENT_UID, deployment_space_id = WML_PRODUCTION_SPACE).result['resources'][0]
asset_deployment_details

{'metadata': {'guid': '6b8d6961-16d2-4c1d-9389-be2c300e8bb7',
  'created_at': '2021-03-29T18:12:34.965Z',
  'modified_at': '2021-03-29T18:12:34.965Z'},
 'entity': {'name': 'predictive attrition - 2018',
  'type': 'online',
  'scoring_endpoint': {'url': 'https://ibm-nginx-svc.zen.svc.cluster.local/ml/v4/deployments/6b8d6961-16d2-4c1d-9389-be2c300e8bb7/predictions'},
  'asset': {},
  'asset_properties': {}}}

In [16]:
model_asset_details_from_deployment=wos_client.service_providers.get_deployment_asset(data_mart_id=DATAMART_ID,service_provider_id=service_provider_id,deployment_id=DEPLOYMENT_UID,deployment_space_id=WML_PRODUCTION_SPACE)

In [17]:
feature_columns = ['POSITION_CODE','DEPARTMENT_CODE','DAYS_WITH_COMPANY','COMMUTE_TIME','AGE_BEGIN_PERIOD','GENDER_CODE','PERIOD_TOTAL_DAYS','STARTING_SALARY','ENDING_SALARY','NB_INCREASES','BONUS','NB_BONUS','VACATION_DAYS_TAKEN','SICK_DAYS_TAKEN','PROMOTIONS','NB_MANAGERS','DAYS_IN_POSITION','DAYS_SINCE_LAST_RAISE','RANKING_CODE','OVERTIME','DBLOVERTIME','TRAVEL']
cat_features = []

['POSITION_CODE',
 'DEPARTMENT_CODE',
 'DAYS_WITH_COMPANY',
 'COMMUTE_TIME',
 'AGE_BEGIN_PERIOD',
 'GENDER_CODE',
 'PERIOD_TOTAL_DAYS',
 'STARTING_SALARY',
 'ENDING_SALARY',
 'NB_INCREASES',
 'BONUS',
 'NB_BONUS',
 'VACATION_DAYS_TAKEN',
 'SICK_DAYS_TAKEN',
 'PROMOTIONS',
 'NB_MANAGERS',
 'DAYS_IN_POSITION',
 'DAYS_SINCE_LAST_RAISE',
 'RANKING_CODE',
 'OVERTIME',
 'DBLOVERTIME',
 'TRAVEL']

In [18]:
subscription_details = wos_client.subscriptions.add(
        data_mart_id=DATAMART_ID,
        service_provider_id=service_provider_id,
        asset=Asset(
            asset_id=model_asset_details_from_deployment["entity"]["asset"]["asset_id"],
            name=model_asset_details_from_deployment["entity"]["asset"]["name"],
            url=model_asset_details_from_deployment["entity"]["asset"]["url"],
            asset_type=AssetTypes.MODEL,
            input_data_type=InputDataType.STRUCTURED,
            problem_type=ProblemType.BINARY_CLASSIFICATION
        ),
        deployment=AssetDeploymentRequest(
            deployment_id=asset_deployment_details['metadata']['guid'],
            name=asset_deployment_details['entity']['name'],
            deployment_type= DeploymentTypes.ONLINE,
            url=asset_deployment_details['entity']['scoring_endpoint']['url']
        ),
        asset_properties=AssetPropertiesRequest(
            label_column="ATTRITION",
            probability_fields=["probability"],
            prediction_field="prediction",
            feature_fields = feature_columns,
            categorical_fields = cat_features,
            training_data_reference=TrainingDataReference(type="cos",
                                                          location=COSTrainingDataReferenceLocation(bucket = "faststartlab-donotdelete-pr-nhfd4jnhlxgpc7",
                                                                                                    file_name = "hr_attrition_data_2019.csv"),
                                                          connection=COSTrainingDataReferenceConnection.from_dict({
                                                                        "resource_instance_id": "crn:v1:bluemix:public:cloud-object-storage:global:a/7d8b3c34272c0980d973d3e40be9e9d2:2883ef10-23f1-4592-8582-2f2ef4973639::",
                                                                        "url": "https://s3.us.cloud-object-storage.appdomain.cloud",
                                                                        "api_key": "yqcPbWZ0AQPHleHVerrR4Wx5e9pymBdMgydbEra5zCif",
                                                                        "iam_url": "https://iam.ng.bluemix.net/oidc/token"}))
        )
    ).result
subscription_id = subscription_details.metadata.id
print('subscription_id: ' + subscription_id)

subscription_id: 2ffd8bc0-494e-47bb-a820-9546194f3442


In [19]:
scoring_data = {
    "input_data": [
        {"fields": feature_columns,
         "values": [
             [3900.0,300.0,60.0,12.0,38.0,0.0,60.0,27500.0,27500.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,60.0,60.0,3.0,0.0,0.0,0.0]
         ]
        }
    ]
}

In [20]:
import time

time.sleep(5)
prediction = wml_client.deployments.score(DEPLOYMENT_UID, scoring_data)
prediction

{'predictions': [{'fields': ['prediction', 'probability'],
   'values': [[0, [0.9986642726171371, 0.0013357273828628622]]]}]}

In [21]:
time.sleep(5)

payload_data_set_id = None
payload_data_set_id = wos_client.data_sets.list(type=DataSetTypes.PAYLOAD_LOGGING, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[0].metadata.id
if payload_data_set_id is None:
    print("Payload data set not found. Please check subscription status.")
else:
    print("Payload data set id:", payload_data_set_id)

Payload data set id: 0551bffa-448e-471f-a3f9-e9b0dfbc72f8


In [22]:
time.sleep(5)
pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
print("Number of records in the payload logging table: {}".format(pl_records_count))
if pl_records_count == 0:
    raise Exception("Payload logging did not happen!")

Number of records in the payload logging table: 1


In [23]:
monitor_instances = wos_client.monitor_instances.list().result.monitor_instances
for monitor_instance in monitor_instances:
    monitor_def_id=monitor_instance.entity.monitor_definition_id
    if monitor_def_id == "drift" and monitor_instance.entity.target.target_id == subscription_id:
        wos_client.monitor_instances.delete(monitor_instance.metadata.id)
        print('Deleted existing drift monitor instance with id: ', monitor_instance.metadata.id)


target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id

)

parameters = {
    "min_samples": 100,
    "drift_threshold": 0.05,
    "train_drift_model": True,
    "enable_model_drift": True,
    "enable_data_drift": True
}

drift_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=DATAMART_ID,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.DRIFT.ID,
    target=target,
    parameters=parameters
).result

drift_monitor_instance_id = drift_monitor_details.metadata.id
drift_monitor_instance_id




 Waiting for end of monitor instance creation 2026e1ce-c869-40e9-8e49-85a62c7438b2 




preparing...........
active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




'2026e1ce-c869-40e9-8e49-85a62c7438b2'

In [24]:
target = Target(
        target_type=TargetTypes.SUBSCRIPTION,
        target_id=subscription_id
)
parameters = {
    "min_feedback_data_size": 50
}
thresholds = [
    {
        "metric_id": "area_under_roc",
        "type": "lower_limit",
        "value": 0.8
    }
]
quality_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=DATAMART_ID,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.QUALITY.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds 
).result
quality_monitor_instance_id = quality_monitor_details.metadata.id
quality_monitor_instance_id




 Waiting for end of monitor instance creation b5501ccc-2268-4fa9-bcbb-369f21f09190 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




'b5501ccc-2268-4fa9-bcbb-369f21f09190'

In [25]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)
parameters = {
    "enabled": True
}
explainability_details = wos_client.monitor_instances.create(
    data_mart_id=DATAMART_ID,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.EXPLAINABILITY.ID,
    target=target,
    parameters=parameters
).result

explainability_monitor_id = explainability_details.metadata.id




 Waiting for end of monitor instance creation eccc32fa-9335-4df3-bfcf-44b74516248d 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




## Feed in historical data

## Create subscription for challenger model

In [26]:
SERVICE_PROVIDER_NAME = "Attrition Pre-Prod Space"
SERVICE_PROVIDER_DESCRIPTION = "Pre-Production space for the HR attrition model"

In [27]:
added_service_provider_result = wos_client.service_providers.add(
        name=SERVICE_PROVIDER_NAME,
        description=SERVICE_PROVIDER_DESCRIPTION,
        service_type=ServiceTypes.WATSON_MACHINE_LEARNING,
        deployment_space_id = WML_PREPROD_SPACE,
        operational_space_id = "pre_production",
        credentials=WML_CREDENTIALS,
        background_mode=False
    ).result
service_provider_id = added_service_provider_result.metadata.id




 Waiting for end of adding service provider d3d91333-80b2-4d7a-a5f7-1bef4a3489ca 




active

-----------------------------------------------
 Successfully finished adding service provider 
-----------------------------------------------




In [28]:
wml_client.set.default_space(WML_PREPROD_SPACE)

'SUCCESS'

In [29]:
DEPLOYMENT_UID = ''
deployments_list = wml_client.deployments.get_details()
for deployment in deployments_list["resources"]:
    if deployment['metadata']['name'] == PREPROD_DEPLOYMENT:
        DEPLOYMENT_UID = deployment['metadata']['id']
print(DEPLOYMENT_UID)

de7807af-91e0-428f-8360-434b5a89e2ea


In [30]:
asset_deployment_details = wos_client.service_providers.list_assets(data_mart_id=DATAMART_ID, service_provider_id=service_provider_id, deployment_id=DEPLOYMENT_UID, deployment_space_id = WML_PREPROD_SPACE).result['resources'][0]
asset_deployment_details

{'metadata': {'guid': 'de7807af-91e0-428f-8360-434b5a89e2ea',
  'created_at': '2021-04-07T04:03:24.392Z',
  'modified_at': '2021-04-07T04:03:24.392Z'},
 'entity': {'name': 'predictive attrition - challenger',
  'type': 'online',
  'scoring_endpoint': {'url': 'https://ibm-nginx-svc.zen.svc.cluster.local/ml/v4/deployments/de7807af-91e0-428f-8360-434b5a89e2ea/predictions'},
  'asset': {},
  'asset_properties': {}}}

In [31]:
model_asset_details_from_deployment=wos_client.service_providers.get_deployment_asset(data_mart_id=DATAMART_ID,service_provider_id=service_provider_id,deployment_id=DEPLOYMENT_UID,deployment_space_id=WML_PREPROD_SPACE)

In [32]:
subscription_details = wos_client.subscriptions.add(
        data_mart_id=DATAMART_ID,
        service_provider_id=service_provider_id,
        asset=Asset(
            asset_id=model_asset_details_from_deployment["entity"]["asset"]["asset_id"],
            name=model_asset_details_from_deployment["entity"]["asset"]["name"],
            url=model_asset_details_from_deployment["entity"]["asset"]["url"],
            asset_type=AssetTypes.MODEL,
            input_data_type=InputDataType.STRUCTURED,
            problem_type=ProblemType.BINARY_CLASSIFICATION
        ),
        deployment=AssetDeploymentRequest(
            deployment_id=asset_deployment_details['metadata']['guid'],
            name=asset_deployment_details['entity']['name'],
            deployment_type= DeploymentTypes.ONLINE,
            url=asset_deployment_details['entity']['scoring_endpoint']['url']
        ),
        asset_properties=AssetPropertiesRequest(
            label_column="ATTRITION",
            probability_fields=["probability"],
            prediction_field="prediction",
            feature_fields = feature_columns,
            categorical_fields = cat_features,
            training_data_reference=TrainingDataReference(type="cos",
                                                          location=COSTrainingDataReferenceLocation(bucket = "faststartlab-donotdelete-pr-nhfd4jnhlxgpc7",
                                                                                                    file_name = "hr_attrition_data_2020.csv"),
                                                          connection=COSTrainingDataReferenceConnection.from_dict({
                                                                        "resource_instance_id": "crn:v1:bluemix:public:cloud-object-storage:global:a/7d8b3c34272c0980d973d3e40be9e9d2:2883ef10-23f1-4592-8582-2f2ef4973639::",
                                                                        "url": "https://s3.us.cloud-object-storage.appdomain.cloud",
                                                                        "api_key": "yqcPbWZ0AQPHleHVerrR4Wx5e9pymBdMgydbEra5zCif",
                                                                        "iam_url": "https://iam.ng.bluemix.net/oidc/token"}))
        )
    ).result
subscription_id = subscription_details.metadata.id
print('subscription_id: ' + subscription_id)

subscription_id: 87ce0142-00ba-44b8-a19c-63b6db5de588


In [33]:
time.sleep(5)
prediction = wml_client.deployments.score(DEPLOYMENT_UID, scoring_data)
prediction

{'predictions': [{'fields': ['prediction', 'probability'],
   'values': [[0, [0.9718800048430234, 0.028119995156976656]]]}]}

In [34]:
time.sleep(5)
payload_data_set_id = None
payload_data_set_id = wos_client.data_sets.list(type=DataSetTypes.PAYLOAD_LOGGING, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[0].metadata.id
if payload_data_set_id is None:
    print("Payload data set not found. Please check subscription status.")
else:
    print("Payload data set id:", payload_data_set_id)

Payload data set id: 916529f6-4d05-49dd-87bb-9a76310d5a77


In [35]:
time.sleep(5)
pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
print("Number of records in the payload logging table: {}".format(pl_records_count))
if pl_records_count == 0:
    raise Exception("Payload logging did not happen!")

Number of records in the payload logging table: 1


In [36]:
monitor_instances = wos_client.monitor_instances.list().result.monitor_instances
for monitor_instance in monitor_instances:
    monitor_def_id=monitor_instance.entity.monitor_definition_id
    if monitor_def_id == "drift" and monitor_instance.entity.target.target_id == subscription_id:
        wos_client.monitor_instances.delete(monitor_instance.metadata.id)
        print('Deleted existing drift monitor instance with id: ', monitor_instance.metadata.id)


target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id

)

parameters = {
    "min_samples": 100,
    "drift_threshold": 0.05,
    "train_drift_model": True,
    "enable_model_drift": True,
    "enable_data_drift": True
}

drift_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=DATAMART_ID,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.DRIFT.ID,
    target=target,
    parameters=parameters
).result

drift_monitor_instance_id = drift_monitor_details.metadata.id
drift_monitor_instance_id




 Waiting for end of monitor instance creation 02e06e2a-c439-4ae6-b0be-e989b6f40d8d 




preparing...........
active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




'02e06e2a-c439-4ae6-b0be-e989b6f40d8d'

In [37]:
target = Target(
        target_type=TargetTypes.SUBSCRIPTION,
        target_id=subscription_id
)
parameters = {
    "min_feedback_data_size": 50
}
thresholds = [
    {
        "metric_id": "area_under_roc",
        "type": "lower_limit",
        "value": 0.8
    }
]
quality_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=DATAMART_ID,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.QUALITY.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds 
).result
quality_monitor_instance_id = quality_monitor_details.metadata.id
quality_monitor_instance_id




 Waiting for end of monitor instance creation 5e4163be-2e5f-4e76-96fc-6324d03cb9c7 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




'5e4163be-2e5f-4e76-96fc-6324d03cb9c7'

In [38]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)
parameters = {
    "enabled": True
}
explainability_details = wos_client.monitor_instances.create(
    data_mart_id=DATAMART_ID,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.EXPLAINABILITY.ID,
    target=target,
    parameters=parameters
).result

explainability_monitor_id = explainability_details.metadata.id




 Waiting for end of monitor instance creation e438928f-927c-4b3e-9d32-9abd43464782 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




In [39]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)
parameters = {
}
mrm_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=DATAMART_ID,
    background_mode=False,
    monitor_definition_id='mrm',
    target=target,
    parameters=parameters
).result

mrm_instance_id = mrm_monitor_details.metadata.id




 Waiting for end of monitor instance creation eec2336d-52e6-4bf0-b8a2-3a9ddbd16ab8 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




## Upload and evaluate test data

In [40]:
!wget https://raw.githubusercontent.com/CloudPak-Outcomes/OpenScale-Data/main/Employee-Attrition/attrition_test_data_2020.csv

--2021-04-07 04:44:42--  https://raw.githubusercontent.com/CloudPak-Outcomes/OpenScale-Data/main/Employee-Attrition/attrition_test_data_2020.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24631 (24K) [text/plain]
Saving to: ‘attrition_test_data_2020.csv’

attrition_test_data 100%[===================>]  24.05K  --.-KB/s    in 0.001s  

2021-04-07 04:44:45 (38.8 MB/s) - ‘attrition_test_data_2020.csv’ saved [24631/24631]



In [42]:
def generate_access_token():
    headers={}
    headers["Accept"] = "application/json"
    auth = HTTPBasicAuth(WOS_CREDENTIALS["username"], WOS_CREDENTIALS["password"])
    
    ICP_TOKEN_URL= WOS_CREDENTIALS["url"] + "/v1/preauth/validateAuth"
    
    response = requests.get(ICP_TOKEN_URL, headers=headers, auth=auth, verify=False)
    json_data = response.json()
    icp_access_token = json_data['accessToken']
    return icp_access_token

In [48]:
print("Running upload and evaluate for test data")
import json
import time
from datetime import datetime

file_name = 'attrition_test_data_2020.csv'

status = None
monitoring_run_id = None
GET_UPLOAD_AND_EVALUATION_STATUS_RETRIES = 32
GET_UPLOAD_AND_EVALUATION_STATUS_INTERVAL = 10

headers = {}
headers["Content-Type"] = "text/csv"
headers["Authorization"] = "Bearer {}".format(generate_access_token())

POST_EVALUATIONS_URL = WOS_CREDENTIALS["url"] + "/openscale/{0}/v2/monitoring_services/mrm/monitor_instances/{1}/risk_evaluations?test_data_set_name={2}".format(DATAMART_ID, mrm_instance_id, file_name)

with open(file_name) as file:
    f = file.read()
    b = bytearray(f, 'utf-8')

response = requests.post(POST_EVALUATIONS_URL, data=bytes(b), headers=headers, verify=False)
if response.ok is False:
    raise Exception("Upload and evalaute for {0} failed with {1}: {2}".format(file_name, response.status_code, response.reason))

headers = {}
headers["Content-Type"] = "application/json"
headers["Authorization"] = "Bearer {}".format(generate_access_token())

GET_EVALUATIONS_URL = WOS_CREDENTIALS["url"] + "/openscale/{0}/v2/monitoring_services/mrm/monitor_instances/{1}/risk_evaluations".format(DATAMART_ID, mrm_instance_id)

for i in range(GET_UPLOAD_AND_EVALUATION_STATUS_RETRIES):

    response = requests.get(GET_EVALUATIONS_URL, headers=headers, verify=False)
    if response.ok is False:
        raise Exception("Getting status of upload and evalaute for {0} failed with {1}: {2}".format(file_name, response.status_code, response.reason))

    response = json.loads(response.text)
    if "metadata" in response and "id" in response["metadata"]:
        monitoring_run_id = response["metadata"]["id"]
    if "entity" in response and "status" in response["entity"]:
        status = response["entity"]["status"]["state"]

    if status is not None:
        print(datetime.utcnow().strftime('%H:%M:%S'), status.lower())
        if status.lower() in ["finished", "completed"]:
            break
        elif "error" in status.lower():
            print(response)
            break

    time.sleep(GET_UPLOAD_AND_EVALUATION_STATUS_INTERVAL)

print(status, monitoring_run_id)

Running upload and evaluate for test data
04:52:36 upload_in_progress
04:52:48 running
04:53:00 running
04:53:15 running
04:53:27 running
04:53:38 running
04:53:50 finished
finished 352309a6-824d-45d6-a13f-c0dc7786b445
